# Criptografia Simétrica

Na criptografia simétrica, o mesmo segredo (chave) é usado tanto para criptografar quanto para descriptografar dados.

#### Principais Características:

- **Chave Única:** Apenas uma chave é utilizada para criptografia e descriptografia.
- **Velocidade:** Geralmente é mais rápida do que a criptografia assimétrica, o que a torna adequada para grandes volumes de dados.
- **Desafios de Compartilhamento:** O principal desafio é o gerenciamento e a segurança da chave secreta, especialmente quando muitos usuários estão envolvidos.
- **Exemplos de Algoritmos:**
 - AES (Advanced Encryption Standard),
 - DES (Data Encryption Standard),
 - 3DES (Triple DES).

#### Aplicações Comuns:

- Criptografia de dados em repouso e em trânsito.
- Proteção de comunicações entre sistemas.
- Armazenamento seguro de dados sensíveis.


# Criptografia Simétrica em Python

Ao contrário de algumas outras linguagens de programação, o Python não possui uma API nativa de criptografia. Vários frameworks de código aberto preenchem essa lacuna. Os pacotes de criptografia mais populares em Python são `cryptography` e `pycryptodome`.

Nas aulas vou utilizar a `cryptography`.

docs: https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/

Instalação: pip install cryptography



In [ ]:
# No Colab já está instalada!
# !pip install cryptography

#### Exemplo utilizado o algoritmo AES e o modo CBC (Cipher Block Chaining)

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import secrets

# Para o modo CBC é necessário o initialization vector (IV)
#iv = secrets.token_bytes(16)
iv = bytes(16)
print("IV (hex):", iv.hex())
print("Tamanho do IV:", len(iv), "byte" )
print("")


# Define a chave (deve ter 128, 192 ou 256 bits para AES)
 # chave de 128 bits - 16 bytes
key = bytes([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4])
#key = b'a chave de 128, 192 ou 256 bits!'

# bytes (n mutavel) ou bytearray (mutavel)
# bytearray permite modificar o vetor de bytes depois de criado,
# enquanto bytes cria vetores de bytes imutáveis.


print("Chave (hex):", key.hex())
print("Tamanho da chave:", len(key), "bytes (",len(key)*8," bits )" )
print("")


# Cria um cifrador AES no modo CBC
cipher = Cipher(
    algorithms.AES(key),       # AES
    modes.CBC(iv),             # Modo CBC com IV
    backend=default_backend()
)


# Cria um objeto de criptografia
encryptor = cipher.encryptor()

# Define o texto em claro (deve ter o tamanho do bloco do AES, que é 16 bytes)
plaintext = b'0123456789ABCDEF'
print("Plaintext (hex): ", plaintext.hex())
print("Tamanho do texto:", len(plaintext), "bytes")
print("")

# Criptografa o texto em claro
ciphertext = encryptor.update(plaintext) + encryptor.finalize()

print("Ciphertext (hex): ", ciphertext.hex())
print("Tamanho do texto cifrado:", len(ciphertext), "bytes")

IV (hex): 00000000000000000000000000000000
Tamanho do IV: 16 byte

Chave (hex): 01010101020202020303030304040404
Tamanho da chave: 16 bytes ( 128  bits )

Plaintext (hex):  30313233343536373839414243444546
Tamanho do texto: 16 bytes

Ciphertext (hex):  acfb0bd63ed23c43913cfea4bdcea766
Tamanho do texto cifrado: 16 bytes


#### Sobre o backend: ` backend=default_backend()`

**`default_backend():`**
Esta função retorna o backend padrão fornecido pela biblioteca cryptography.
Oermite que o pacote cryptography seja flexível e extensível, já que diferentes backends podem ser implementados para diferentes necessidades.

#### Decifrando

In [ ]:
# Cria um objeto de criptografia
# cipher já foi criado antes!
decryptor = cipher.decryptor()

# Descriptografando o texto cifrado
plaintext_dec = decryptor.update(ciphertext) + decryptor.finalize()

print(plaintext_dec.hex())

30313233343536373839414243444546


**encryptor.update(plaintext):** Criptografa o texto em claro, mas pode não processar todos os dados se o tamanho do texto não for um múltiplo do tamanho do bloco.


**encryptor.finalize()**: Finaliza o processo de criptografia, aplicando o preenchimento necessário e retornando quaisquer dados adicionais que precisam ser adicionados ao texto cifrado.

#### E se o tamanho do texto não for múltiplo do tamanho do bloco? Padding!



In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
import secrets

# Para o modo CBC é necessário o initialization vector (IV)
#iv = secrets.token_bytes(16)
iv = bytes(16)
print("IV (hex):", iv.hex())
print("Tamanho do IV:", len(iv), "byte" )
print("")


# Define a chave (deve ter 128, 192 ou 256 bits para AES)
key = bytes([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]) # chave de 128 bits - 8 bytes
#key = b'a chave de 128, 192 ou 256 bits!'

# bytes ou bytearray?
# bytearray permite modificar o vetor de bytes depois de criado, enquanto bytes cria vetores de bytes imutáveis.


print("Chave (hex):", key.hex())
print("Tamanho da chave:", len(key), "bytes (",len(key)*8," bits )" )
print("")


# Cria um cifrador AES no modo CBC
cipher = Cipher(
    algorithms.AES(key),       # AES
    modes.CBC(iv),             # Modo CBC com IV
    backend=default_backend()
)


# Cria um objeto de criptografia
encryptor = cipher.encryptor()

# Define o texto em claro (deve ter o tamanho do bloco do AES, que é 16 bytes)
# Note que se estiver sem faltar nenhum ele vai colocar padding, pois se tem paggin tem padding!!!!
# pois se não faltar e terminar em 01 faz oq? se não tem padding para caso não falte padding?
plaintext = b'01234567891234'
print("Plaintext (hex): ", plaintext.hex())
print("Tamanho do texto:", len(plaintext), "bytes")
print("")


# Preenchimento do texto em claro para ajustar ao tamanho do bloco
padder = padding.PKCS7(algorithms.AES.block_size).padder()
padded_plaintext = padder.update(plaintext) + padder.finalize()

print("Plaintext com padding (hex): ", padded_plaintext.hex())
print("Tamanho do texto com Padding:", len(padded_plaintext), "bytes")
print("")


# Criptografa o texto em claro
ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()

print("Ciphertext (hex): ", ciphertext.hex())
print("Tamanho do texto cifrado:", len(ciphertext), "bytes")

IV (hex): 00000000000000000000000000000000
Tamanho do IV: 16 byte

Chave (hex): 01010101020202020303030304040404
Tamanho da chave: 16 bytes ( 128  bits )

Plaintext (hex):  30313233343536373839313233343541
Tamanho do texto: 16 bytes

Plaintext com padding (hex):  3031323334353637383931323334354110101010101010101010101010101010
Tamanho do texto com Padding: 32 bytes

Ciphertext (hex):  702d7cdc646a5e17fbaeec401d0784129ee6f654f27e3b0b5d268660f284f559
Tamanho do texto cifrado: 32 bytes


#### Decifrando e ajustando padding

In [ ]:
# Cria um objeto de criptografia
# cipher já foi criado antes!
decryptor = cipher.decryptor()

# Descriptografando o texto cifrado
plaintext_dec = decryptor.update(ciphertext) + decryptor.finalize()

print("Plaintext com padding (hex): ", plaintext_dec.hex())

# Remoção do preenchimento
unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
plaintext_unpadded = unpadder.update(plaintext_dec) + unpadder.finalize()

print("Plaintext (hex): ", plaintext_unpadded.hex())



Plaintext com padding (hex):  30313233343536373839060606060606
Plaintext (hex):  30313233343536373839


#### E se o padding estiver incorreto?

In [ ]:
# Cria um objeto de criptografia
# cipher já foi criado antes!
decryptor = cipher.decryptor()

# Descriptografando o texto cifrado
plaintext_dec = decryptor.update(ciphertext) + decryptor.finalize()

# Modificando o Padding
plaintext_dec= bytearray(plaintext_dec)
plaintext_dec[15] = 0

print("Plaintext com padding (hex): ", plaintext_dec.hex())

# Remoção do preenchimento
unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
plaintext_unpadded = unpadder.update(plaintext_dec) + unpadder.finalize()

print("Plaintext (hex): ", plaintext_unpadded.hex())


## Exemplo para testes

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

def encrypt_aes(key: bytes, iv: bytes, plaintext: bytes) -> bytes:
    """
    Criptografa o texto usando AES no modo CBC.

    :param key: Chave de criptografia de 16, 24 ou 32 bytes.
    :param iv: Vetor de inicialização de 16 bytes.
    :param plaintext: Texto em claro a ser criptografado.
    :return: Texto cifrado.
    """
    # Criação do cifrador AES no modo CBC
    cipher = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    )

    # Criando o objeto de criptografia
    encryptor = cipher.encryptor()

    # Preenchimento do texto em claro para ajustar ao tamanho do bloco
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()

    # Criptografando o texto em claro
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()

    return ciphertext

def decrypt_aes(key: bytes, iv: bytes, ciphertext: bytes) -> bytes:
    """
    Descriptografa o texto cifrado usando AES no modo CBC.

    :param key: Chave de criptografia de 16, 24 ou 32 bytes.
    :param iv: Vetor de inicialização de 16 bytes.
    :param ciphertext: Texto cifrado a ser descriptografado.
    :return: Texto em claro.
    """
    # Criação do cifrador AES no modo CBC
    cipher = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    )

    # Criando o objeto de descriptografia
    decryptor = cipher.decryptor()

    # Descriptografando o texto cifrado
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    # Remoção do preenchimento
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plaintext = unpadder.update(padded_plaintext) + unpadder.finalize()

    return plaintext

##################
# Exemplo de uso #
##################

key = b'0123456789abcdee'  # Chave de 16 bytes
iv = b'1234567890abcdef'  # IV de 16 bytes
plaintext = b'Hello World!'  # Texto em claro
print("Texto em claro (hex):", plaintext.hex())


# Criptografar
ciphertext = encrypt_aes(key, iv, plaintext)
print("Texto cifrado (hex):", ciphertext.hex())

# Descriptografar
decrypted_text = decrypt_aes(key, iv, ciphertext)
print("Texto descriptografado (hex):", decrypted_text.hex())
print("Texto descriptografado (String):", decrypted_text.decode())

Texto em claro (hex): 48656c6c6f20576f726c6421
Texto cifrado (hex): 9d390be35c1798a9c0a181caa86758b9
Texto descriptografado (hex): 48656c6c6f20576f726c6421
Texto descriptografado (String): Hello World!


# Exercício 1: Cifrando e Decifrando Arquivos com Cabeçalho Personalizado

**Objetivo:**  
Criar um programa em Python capaz de criptografar e descriptografar arquivos, utilizando AES no modo CBC e um cabeçalho de 32 bytes com metadados sobre o arquivo cifrado.

---

## Instruções

### 1. Cabeçalho do Arquivo Cifrado (32 bytes)

O início do arquivo criptografado deve conter um cabeçalho com os seguintes campos:

| Campo         | Tamanho (bytes) | Descrição                                                                 |
|---------------|-----------------|---------------------------------------------------------------------------|
| Identificador | 2               | Deve conter uma sequência fixa, ex: `b'ED'`, para indicar um arquivo cifrado |
| Versão        | 1               | Versão do formato de cabeçalho (ex: `1`)                                  |
| Algoritmo     | 1               | `1` para AES (reservado para futuras extensões com outros algoritmos)     |
| Modo          | 1               | `1` para modo CBC                                                         |
| IV            | 16              | Vetor de inicialização (gerado aleatoriamente na criptografia)           |
| Reserved      | 11              | Reservado para uso futuro (preencher com `0x00`)                          |

---

### 2. Etapas do Programa (Encrypt)

1. Solicitar ao usuário o caminho de um arquivo para criptografar.  
2. Gerar uma chave de 256 bits (pode ser fixa ou pedida ao usuário).  
3. Gerar o IV (Initialization Vector) aleatoriamente.  
4. Criar o cabeçalho conforme especificado.  
5. Criptografar o conteúdo do arquivo usando AES-CBC.  
6. Escrever o cabeçalho + dados criptografados em um novo arquivo.
7. Salvar o arquivo concatenando ".enc" no final do nome do arquivo.

---

### 3. Descriptografia (Decrypt)

1. Ler o cabeçalho do arquivo cifrado.  
2. Verificar se o identificador, versão e algoritmo estão corretos (validação).  
3. Extrair o IV.  
4. Usar a chave (a mesma da criptografia) para decifrar o conteúdo.  
5. Salvar o arquivo original.

---

## Dicas

- Use o pacote `cryptography` com `Cipher`, `algorithms.AES`, `modes.CBC` e `padding.PKCS7`.
- Lembre-se de aplicar e remover o padding adequadamente.
- Para escrever e ler os campos binários, use a biblioteca `struct` ou concatene os bytes com cuidado.


# Exercício 2: Gerando Arquivo de Metadados para Verificação de Integridade

**Objetivo:**  
Criar um programa em Python que não altera o arquivo original, mas gera um **arquivo de metadados** (48 bytes) contendo um cabeçalho e um fingerprint de 16 bytes (último bloco do AES‑CBC) para verificar se o arquivo original foi modificado.

---

## Instruções

### 1. Cabeçalho do Arquivo de Metadados (48 bytes)

O arquivo de metadados deve conter exatamente 48 bytes, dispostos assim:

| Campo         | Tamanho (bytes) | Descrição                                                                                 |
|---------------|-----------------|-------------------------------------------------------------------------------------------|
| Identificador | 2               | Sequência fixa, ex: `b'CF'`, para indicar “Crypto Fingerprint”                            |
| Versão        | 1               | Versão do formato (ex: `1`)                                                               |
| Algoritmo     | 1               | `1` para AES                                                                              |
| Modo          | 1               | `1` para CBC                                                                              |
| IV            | 16              | Vetor de inicialização (aleatório)                                                        |
| Fingerprint   | 16              | Último bloco do ciphertext gerado a partir do arquivo original (garante integridade)      |
| Reserved      | 11              | Reservado para uso futuro (preencher com `0x00`)                                          |

---

### 2. Geração do Arquivo de Metadados

1. **Entrada do Usuário**  
   - Solicitar o caminho do arquivo original a ser protegido.

2. **Chave e IV**  
   - Gerar (ou solicitar) uma chave AES de 256 bits.  
   - Gerar um IV aleatório de 16 bytes.

3. **Cálculo do Fingerprint**  
   - Ler todo o conteúdo do arquivo original.  
   - Aplicar AES‑CBC com padding PKCS7 sobre esse conteúdo **somente em memória** (não salvar o ciphertext em disco).  
   - Extrair os **últimos 16 bytes** do ciphertext gerado como fingerprint.

4. **Montagem do Cabeçalho**  
   - Empacotar, em ordem:  
     1. Identificador (2 bytes)  
     2. Versão (1 byte)  
     3. Algoritmo (1 byte)  
     4. Modo (1 byte)  
     5. IV (16 bytes)  
     6. Fingerprint (16 bytes)  
     7. Reserved (11 bytes de `0x00`)

5. **Gravação do Arquivo de Metadados**  
   - Salvar esses 48 bytes em um novo arquivo (por exemplo, com extensão `.meta`).

---

### 3. Verificação de Integridade

1. **Leitura do Arquivo de Metadados**  
   - Extrair cada campo do cabeçalho e validar.

2. **Reprodução do Fingerprint**  
   - Ler o mesmo arquivo original.  
   - Executar AES‑CBC in‑memory com a chave e IV extraídos.  
   - Extrair os últimos 16 bytes do ciphertext.

3. **Comparação**  
   - Se o fingerprint recalculado for **igual** ao fingerprint armazenado no cabeçalho, o arquivo **não foi alterado**.  
   - Caso contrário, sinalizar “Arquivo modificado ou corrompido”.

---

## Dicas
- Use `cryptography` para `Cipher`, `algorithms.AES`, `modes.CBC` e `padding.PKCS7`.  



In [ ]:
# Define campos
IDENT       = b'CF'                 # 2 bytes
VERSION     = bytes([0x01])         # 1 byte
ALGO        = bytes([0x01])         # 1 byte (AES)
MODE        = bytes([0x01])         # 1 byte (CBC)
IV          = bytes(range(0x00, 0x10))     # 16 bytes: 0x00..0x0F
FINGERPRINT = bytes(range(0x10, 0x20))     # 16 bytes: 0x10..0x1F
RESERVED    = bytes(11)             # 11 bytes de 0x00

# Monta o header (48 bytes)
header = bytearray()
header += IDENT
header += VERSION
header += ALGO
header += MODE
header += IV
header += FINGERPRINT
header += RESERVED

# Salva no arquivo
with open("arquivo.meta", "wb") as f:
    f.write(header)

In [ ]:
# Lê do arquivo e separa os campos com slicing
with open("arquivo.meta", "rb") as f:
    data = f.read(48)

ident      = data[0:2]        # 2 bytes
version    = data[2]          # 1 byte
algo       = data[3]          # 1 byte
mode       = data[4]          # 1 byte
iv         = data[5:21]       # 16 bytes
fingerprint= data[21:37]      # 16 bytes
reserved   = data[37:48]      # 11 bytes

# ✅ Imprime os dados
print("Identificador:", ident, ident.decode())
print("Versão:", version)
print("Algoritmo:", algo)
print("Modo:", mode)
print("IV:", iv.hex())
print("Fingerprint:", fingerprint.hex())
print("Reserved:", reserved.hex())

Identificador: b'CF' CF
Versão: 1
Algoritmo: 1
Modo: 1
IV: 000102030405060708090a0b0c0d0e0f
Fingerprint: 101112131415161718191a1b1c1d1e1f
Reserved: 0000000000000000000000
